<a href="https://colab.research.google.com/github/nausheen53/SMAI/blob/master/Q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import ssl
import sklearn
import numpy as np
import spacy
import re
import gensim
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
import textblob
from textblob import TextBlob, Word
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [5]:

df = pd.read_csv('/content/drive/My Drive/Train.csv',header = 0)
print(len(df))

15663


In [0]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [8]:
train = np.array(train)
test = np.array(test)
labels_of_train=[]
for row in train:
    labels_of_train.append(row[2])
text_of_train = []
for row in train:
    text_of_train.append(row[1])
print(len(labels_of_train))    

labels_of_test=[]
for row in test:
    labels_of_test.append(row[2])
text_of_test = []
for row in test:
    text_of_test.append(row[1])

12530


In [0]:
import string
from collections import Counter
final_list_train=[]
for sent in text_of_train:
    # print("1")
    sent = sent.lower()
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    # print("sent1 ",sent)
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(sent)
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    #remove stopwords
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    sent=" ".join(words)
    # print("sentttt ",sent)
    final_list_train.append(sent)
    
    

In [0]:
import string
from collections import Counter
final_list_test=[]
for sent in text_of_test:
    # print("1")
    sent = sent.lower()
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    # print("senttttt ",sent)
#     tokens= sent.split()
    ###################################################
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(sent)
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    #remove stopwords
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    sent=" ".join(words)
    final_list_test.append(sent)

In [20]:
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [21]:
print(model.vector_size)

300


In [23]:
li=[]
li1=[]
i=0
for stri in final_list_train:
  words1 = stri.split()
  doc = [model[word] for word in words1 if word in model.vocab]
  print(len(doc))
  if(len(doc)==0):
    li1.append(i)
  else:
    vector_list = np.mean(doc,axis=0)
    li.append(vector_list)
  i=i+1
  

      

print(len(li))
print(len(li[1]))

Streaming output truncated to the last 5000 lines.
11
10
8
9
17
8
16
16
13
4
4
17
17
10
10
9
24
23
7
12
8
7
13
5
10
13
11
9
12
4
19
11
17
15
31
7
19
9
17
7
7
7
3
5
32
13
46
9
3
5
10
7
19
12
3
21
18
4
22
28
14
15
10
13
9
11
5
9
16
6
6
16
6
30
24
5
19
4
21
22
17
14
5
9
8
16
9
3
10
8
37
11
9
8
30
38
7
7
6
15
6
11
7
16
3
10
4
61
3
7
8
18
3
22
24
2
29
5
8
6
3
40
34
15
8
4
13
13
5
8
3
12
3
5
2
9
24
8
19
7
18
9
18
22
2
5
28
17
15
9
6
3
29
11
13
22
8
17
4
24
24
7
14
4
22
4
10
19
33
4
6
11
9
9
9
14
34
6
5
6
14
9
20
7
8
13
20
15
19
18
7
13
9
3
11
15
4
39
14
14
11
8
13
13
15
5
12
22
8
6
14
4
10
8
3
7
15
7
6
14
3
8
6
20
9
12
19
10
12
15
8
26
10
18
9
9
24
10
10
16
4
28
5
11
3
4
6
9
24
7
4
9
8
24
24
18
5
17
1
9
11
8
11
9
10
24
1
13
19
5
26
14
11
14
1
18
4
10
24
22
4
11
10
11
3
24
8
3
2
5
9
7
17
4
9
15
14
17
21
25
16
13
4
3
2
14
8
20
17
12
6
17
7
11
11
19
3
11
4
11
19
2
18
11
8
7
12
14
5
11
10
11
3
8
23
16
39
9
11
19
34
12
11
6
5
6
8
10
6
9
14
9
11
10
7
13
4
9
3
14
9
25
6
20
10
14
13
7
17
6
7
12
9
19

In [24]:
train_set_final=np.array([np.array(xi) for xi in li])
print(train_set_final.shape)

(12523, 300)


In [25]:
print(len(labels_of_train))
for i in li1:
  print(i)
  del labels_of_train[i]
print(len(labels_of_train))

12530
5076
9522
10348
11005
11938
12341
12458
12523


In [26]:
lis=[]
lis1=[]
i=0
for stri in final_list_test:
  # print(stri)
  words1 = stri.split()
  # print(words1)
  # print(i)
  vector_list = [model[word] for word in words1 if word in model.vocab]
  # words_filtered = [word for word in words1 if word in model.vocab]
  # # Zip the words together with their vector representations
  # word_vec_zip = zip(words_filtered, vector_list)
  # # print((word_vec_zip))
  # word_vec_dict = dict(word_vec_zip)
  # df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
  # # print(df)
  if(len(vector_list)!=0):
    vector_list = np.mean(vector_list, axis=0)
    lis.append(vector_list)
  else:
    lis1.append(i)
  i=i+1
  
print(len(lis1)) 

3


In [27]:
test_set_final=np.array([np.array(xi) for xi in lis])
print(test_set_final.shape)
print(len(labels_of_test))
for i in lis1:
  print(i)
  del labels_of_test[i]
print(len(labels_of_test))

(3130, 300)
3133
810
2313
2875
3130


In [28]:
clf = svm.LinearSVC(C=2.0)
print((train_set_final).shape)
print(len(labels_of_train))
print(len(labels_of_train))
clf.fit(train_set_final,labels_of_train)
print(clf.predict(test_set_final))
predicted_label=clf.predict(test_set_final)

(12523, 300)
12523
12523
['HPL' 'MWS' 'MWS' ... 'EAP' 'HPL' 'MWS']


In [0]:
accu_score = accuracy_score(labels_of_test, predicted_label,normalize=True)

In [30]:
print(accu_score)

0.6840255591054313


In [0]:
f11 = f1_score(labels_of_test, predicted_label,average=None)
re = recall_score(labels_of_test, predicted_label,average='weighted')
cm = confusion_matrix(labels_of_test, predicted_label)
pr_sc = precision_score(labels_of_test, predicted_label,average='weighted')
clrp = classification_report(labels_of_test, predicted_label)
print("accuracy score ",accu_score)
print("********************")
print("f1 score ",f11)
print("********************")
print("recall score ",re)
print("********************")
print("confusion matrix ",cm)
print("********************")
print("precision score ",pr_sc)
print("********************")
print("classification report ",clrp)

accuracy score  0.6824281150159744
********************
f1 score  [0.70075188 0.66329399 0.67435475]
********************
recall score  0.6824281150159744
********************
confusion matrix  [[932 173 186]
 [228 590  99]
 [209  99 614]]
********************
precision score  0.6825086398424383
********************
classification report                precision    recall  f1-score   support

         EAP       0.68      0.72      0.70      1291
         HPL       0.68      0.64      0.66       917
         MWS       0.68      0.67      0.67       922

    accuracy                           0.68      3130
   macro avg       0.68      0.68      0.68      3130
weighted avg       0.68      0.68      0.68      3130

